In [ ]:
import dasimulator as das
import carla
import asyncio
import numpy as np
import pandas as pd
import scipy.fft as fft
import matplotlib.pyplot as plt

## Connect to carla server

In [ ]:
# Carla's world parameters
dt = 0.005
sync = True
render = False
carla_port = 2000
carla_town = 'Town04'

# Vehicle used for the simulation
vehicle_model = 'vehicle.tesla.model3'

In [ ]:
# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', carla_port) 
client.set_timeout(12.0) 

world = client.get_world()
# Apply recommended settings
settings = world.get_settings()
settings.no_rendering_mode = not render
settings.fixed_delta_seconds = dt
settings.synchronous_mode = sync
world.apply_settings(settings)
world = client.load_world(carla_town, reset_settings=False)

#### Load FTP-75 cycle speed profile

In [ ]:
df = pd.read_csv('FTP-75-cycle/FTP-75.csv')
df.drop(index=0, inplace=True, axis=0)
df = df.dropna(axis='index')
df['Phase'] = df['Phase'].astype(int)
df = df[(df['Phase'] == 1) | (df['Phase'] == 2)]
df.drop('Phase', inplace=True, axis=1)

ftp_v = np.array(df['Set Spd'], dtype=float)
ftp_t = np.array(df['Time'], dtype=float)
# converting from mph to km/h
ftp_v = 1.60934 * ftp_v

ftp_duration = ftp_t[-1]

#### Spawn the vehicle 

In [ ]:
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 
vehicle = world.try_spawn_actor(bp_lib.find(vehicle_model), spawn_points[134])

##### Define a trajectory

In [ ]:
# Define the route as a list of spawn points, then convert it to a list of carla waypoints
route_ids = [75,100,92,272,134,138,130,296,222,212,139,352,336]
waypoints = [spawn_points[idx] for idx in route_ids]

# Draw the spawn points locations that are part of the route (useful only if rednder=True for visualizing the route) 
for idx in route_ids:
    world.debug.draw_string(spawn_points[idx].location, str(idx), life_time=15, color=carla.Color(255,0,0))

## Define the Driver Aggressiveness Simulator object

In [ ]:
filename = 'data_collected.csv'
gen = das.DASimulator(world, vehicle, waypoints, target_aggIn=80, opt_dict={'ignore_stop_signs' : True, 'ignore_traffic_lights' : True})

### Start the simulation

In [ ]:
await gen.sim(max_duration=ftp_duration, filename=filename, speed_profile=ftp_v, stop_at_end_pos=False)

## Visualize the data collected

In [ ]:
signals_df = pd.read_csv(filename, sep=',', header=0)
x_t = np.array(signals_df['Time'], dtype=float)
y_rv = np.array(signals_df['Velocity'], dtype=float)
y_tv = np.array(signals_df['Target Velocity'], dtype=float)
y_tp = np.array(signals_df['Throttle'], dtype=float)
y_bp = np.array(signals_df['Brake'], dtype=float)
acc_x = np.array(signals_df['AccX'], dtype=float)
acc_y = np.array(signals_df['AccY'], dtype=float)
acc_z = np.array(signals_df['AccZ'], dtype=float)
gyro_x = np.array(signals_df['GyroX'], dtype=float)
gyro_y = np.array(signals_df['GyroY'], dtype=float)
gyro_z = np.array(signals_df['GyroZ'], dtype=float)

In [ ]:
# Limit parameters for plotting
x_a = 0         # default
x_b = x_t[-1]   # default

plt.figure(figsize=(30,6))
plt.title("Real speed vs Target speed")
plt.ylabel("Speed [km/h]")
plt.xlabel("Time [s]")
plt.xlim(x_a, x_b)
plt.plot(x_t, y_tv, label='Target Vehicle Speed')
plt.plot(x_t, y_rv, label='Real Vehicle Speed')
plt.legend(loc="lower right")
plt.show()

plt.figure(figsize=(30,6))
plt.title("Throttle/Brake")
plt.ylabel("[%]")
plt.xlabel("Time [s]")
plt.xlim(x_a, x_b)
plt.plot(x_t, 100 * y_tp, label='Throttle', color='green')
plt.plot(x_t, -100 * y_bp, label='Brake', color='orange')
plt.legend(loc="lower right")
plt.show()

plt.figure(figsize=(30,6))
plt.title("Subsampled IMU Acceleration (40 Hz)")
plt.ylabel("Acceleration [m/s^2]")
plt.xlabel("Time [s]")
plt.xlim(x_a, x_b)
plt.plot(x_t[::5], acc_y[::5], label='AccY', color='orange')
plt.plot(x_t[::5], acc_x[::5], label='AccX', color='green')
plt.legend(loc="lower right")
plt.show()

## Calculate the actual Aggressiveness Index

Recalling that the index defined in the reference paper is the following:
$\Large{AggIn=\frac{avg(ESD(TP(t)))}{std(v_{desired}(t)-v_{real}(t))}}$

Where the Energy Spectral Density for descrete signal is given by $\large{\bar{S}_{x x}(f)=\lim _{N \rightarrow \infty}(\Delta t)^2\left|\sum_{n=-N}^N x_n e^{-i 2 \pi f n \Delta t}\right|^2}$

In [ ]:
# Define the function for computing the Energy Spectral Density of a signal

def esd(yt, fs):
    """Computes the Energy Spectral Density of a signal yt, keeping only positive frequencies (note that this means that the total energy is halved). 
    
    Parameters
    ----------
        yt : array_like 
            Signal to be analyzed
        fs : float 
            Sampling frequency of the signal

    Returns
    -------
        out : array_like, array_like
            Frequency axis and Energy Spectral Density of the signal
    """
    N = len(yt)
    f = fft.rfftfreq(N, 1/fs)
    Sxx = np.square(1 / fs) *  np.square(np.abs(fft.rfft(yt)))
    
    return f, Sxx

In [ ]:
fs_FTP = 10 # Hz (sampling frequency of the FTP-75 cycle)
k_subsample = int(1 / (dt * fs_FTP)) # subsampling factor for the collected data (to match fs_FTP)

f, Sxx = esd(y_tp[::k_subsample], fs_FTP)
print("Mean ESD: ", np.mean(Sxx))
print("Std of tracking error: ", np.std(y_tv - y_rv))
print( "Aggressiveness Index: ", np.mean(Sxx) / np.std(y_tv - y_rv))